In [1]:
import pandas as pd
import numpy as np

from keras import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from keras.optimizers import SGD, Adam
from keras.losses import MeanSquaredError, BinaryCrossentropy

import optuna

c:\Users\yveem\miniconda3\envs\machine_learning\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### **Getting training & validation data**

In [2]:
df_train = pd.read_csv('../databases/training.csv')
df_valid = pd.read_csv('../databases/validation.csv')

In [3]:
X_train, y_train = df_train.drop(['IND_BOM_1_1'], axis=1), df_train['IND_BOM_1_1']
X_val, y_val = df_valid.drop(['IND_BOM_1_1'], axis=1), df_valid['IND_BOM_1_1']

In [4]:
X_train = np.array(X_train)
X_val = np.array(X_val)

#### **Parameters selection**

In [5]:
params = {
    'hidden_layer_units': [32, 128, 256],
    'hidden_layers': {
        'low': 1,
        'high': 2
    },
    'alpha': [0.0001, 0.01],
    'max_iter': [2000, 5000, 10000],
    'batch_size': [32, 64, 128],
    'activation': ['tanh', 'relu', 'sigmoid'],
    'optimizer': ['SGD', 'Adam'],
    'output_activation': ['sigmoid', 'softmax'],
    'loss_function': ['binary_crossentropy', 'mse']    
}

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    verbose=1,
    restore_best_weights=True,
    min_delta=0.001
)

def objective(trial):
    model = Sequential()
    input_dimension = X_train.shape[1]
    
    model.add(
        Dense(
            input_dim=input_dimension,
            units=trial.suggest_categorical(
                'hidden_layer_units',
                params['hidden_layer_units']
            ),
            activation=trial.suggest_categorical(
                'activation',
                params['activation']
            )
        )
    )

    hidden_layers = trial.suggest_int('hidden_layers', params['hidden_layers']['low'], params['hidden_layers']['high'])
    
    for _ in range(hidden_layers):
        model.add(
            Dense(
                units=trial.suggest_categorical(
                    'hidden_layer_units',
                    params['hidden_layer_units']
                ),
                activation=trial.suggest_categorical(
                    'activation',
                    params['activation']
                )
            )
        )
    
    model.add(
        Dense(
            units=1,
            activation=trial.suggest_categorical(
                'output_activation',
                params['output_activation']
            )
        )
    )

    alpha = trial.suggest_categorical('alpha', params['alpha'])
    optimizer_name = trial.suggest_categorical('optimizer', params['optimizer'])

    if optimizer_name == 'SGD':
        optimizer = SGD(learning_rate=alpha)
    elif optimizer_name == 'Adam':
        optimizer = Adam(learning_rate=alpha)

    model.compile(
        optimizer=optimizer,
        loss=trial.suggest_categorical('loss_function', params['loss_function']),
        metrics=['accuracy']
    )

    model.fit(
        X_train,
        y_train,
        batch_size=trial.suggest_categorical(
            'batch_size',
            params['batch_size']
        ),
        epochs=trial.suggest_categorical(
            'max_iter',
            params['max_iter']
        ),
        validation_data=(X_val, y_val),
        callbacks=[early_stopping]
    )

    loss, accuracy = model.evaluate(X_val, y_val)

    return loss

In [6]:
n_trials = 10

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=n_trials)

[I 2023-03-29 21:21:53,923] A new study created in memory with name: no-name-ed646e20-eb50-4675-bd92-317946ed65e4


Epoch 1/5000
3953/3953 [==============================] - 16s 4ms/step - loss: 0.6516 - accuracy: 0.6541 - val_loss: 0.6465 - val_accuracy: 0.6554
Epoch 2/5000
3953/3953 [==============================] - 14s 3ms/step - loss: 0.6449 - accuracy: 0.6555 - val_loss: 0.6442 - val_accuracy: 0.6554
Epoch 3/5000
3953/3953 [==============================] - 14s 3ms/step - loss: 0.6430 - accuracy: 0.6555 - val_loss: 0.6424 - val_accuracy: 0.6554
Epoch 4/5000
3953/3953 [==============================] - 14s 3ms/step - loss: 0.6413 - accuracy: 0.6555 - val_loss: 0.6408 - val_accuracy: 0.6554
Epoch 5/5000
3953/3953 [==============================] - 14s 3ms/step - loss: 0.6398 - accuracy: 0.6555 - val_loss: 0.6394 - val_accuracy: 0.6554
Epoch 6/5000
3953/3953 [==============================] - 14s 3ms/step - loss: 0.6384 - accuracy: 0.6555 - val_loss: 0.6380 - val_accuracy: 0.6554
Epoch 7/5000
3953/3953 [==============================] - 14s 3ms/step - loss: 0.6371 - accuracy: 0.6555 - val_loss: 0

[I 2023-03-29 21:45:05,350] Trial 0 finished with value: 0.6074244976043701 and parameters: {'hidden_layer_sizes': 256, 'activation': 'relu', 'hidden_layers': 2, 'output_activation': 'sigmoid', 'alpha': 0.0001, 'optimizer': 'SGD', 'loss_function': 'binary_crossentropy', 'batch_size': 64, 'max_iter': 5000}. Best is trial 0 with value: 0.6074244976043701.


Epoch 1/2000
1977/1977 [==============================] - 6s 3ms/step - loss: 0.2106 - accuracy: 0.6679 - val_loss: 0.2075 - val_accuracy: 0.6719
Epoch 2/2000
1977/1977 [==============================] - 6s 3ms/step - loss: 0.2064 - accuracy: 0.6750 - val_loss: 0.2067 - val_accuracy: 0.6740
Epoch 3/2000
1977/1977 [==============================] - 6s 3ms/step - loss: 0.2057 - accuracy: 0.6770 - val_loss: 0.2064 - val_accuracy: 0.6748
Epoch 4/2000
1977/1977 [==============================] - 6s 3ms/step - loss: 0.2050 - accuracy: 0.6782 - val_loss: 0.2061 - val_accuracy: 0.6744
Epoch 5/2000
1977/1977 [==============================] - 6s 3ms/step - loss: 0.2044 - accuracy: 0.6791 - val_loss: 0.2052 - val_accuracy: 0.6771
Epoch 6/2000
1977/1977 [==============================] - 6s 3ms/step - loss: 0.2037 - accuracy: 0.6807 - val_loss: 0.2048 - val_accuracy: 0.6780
Epoch 7/2000
1977/1977 [==============================] - 7s 4ms/step - loss: 0.2032 - accuracy: 0.6817 - val_loss: 0.2043 -

[I 2023-03-29 21:47:38,665] Trial 1 finished with value: 0.20302513241767883 and parameters: {'hidden_layer_sizes': 128, 'activation': 'tanh', 'hidden_layers': 1, 'output_activation': 'sigmoid', 'alpha': 0.0001, 'optimizer': 'Adam', 'loss_function': 'mse', 'batch_size': 128, 'max_iter': 2000}. Best is trial 1 with value: 0.20302513241767883.


...

[I 2023-03-29 22:16:19,896] Trial 7 finished with value: 0.34456583857536316 and parameters: {'hidden_layer_sizes': 32, 'activation': 'relu', 'hidden_layers': 1, 'output_activation': 'softmax', 'alpha': 0.01, 'optimizer': 'SGD', 'loss_function': 'mse', 'batch_size': 64, 'max_iter': 5000}. Best is trial 1 with value: 0.20302513241767883.


Epoch 1/10000
1977/1977 [==============================] - 7s 3ms/step - loss: 0.2314 - accuracy: 0.6539 - val_loss: 0.2302 - val_accuracy: 0.6546
Epoch 2/10000
1977/1977 [==============================] - 7s 4ms/step - loss: 0.2293 - accuracy: 0.6552 - val_loss: 0.2285 - val_accuracy: 0.6553
Epoch 3/10000
1977/1977 [==============================] - 7s 3ms/step - loss: 0.2279 - accuracy: 0.6554 - val_loss: 0.2274 - val_accuracy: 0.6554
Epoch 4/10000
1977/1977 [==============================] - 6s 3ms/step - loss: 0.2269 - accuracy: 0.6554 - val_loss: 0.2266 - val_accuracy: 0.6554
Epoch 5/10000
1977/1977 [==============================] - 7s 3ms/step - loss: 0.2263 - accuracy: 0.6555 - val_loss: 0.2261 - val_accuracy: 0.6554
Epoch 6/10000
1977/1977 [==============================] - 7s 3ms/step - loss: 0.2259 - accuracy: 0.6555 - val_loss: 0.2257 - val_accuracy: 0.6554
Epoch 7/10000
1977/1977 [==============================] - 8s 4ms/step - loss: 0.2256 - accuracy: 0.6555 - val_loss: 0

[I 2023-03-29 22:18:43,721] Trial 8 finished with value: 0.22506356239318848 and parameters: {'hidden_layer_sizes': 128, 'activation': 'relu', 'hidden_layers': 2, 'output_activation': 'sigmoid', 'alpha': 0.0001, 'optimizer': 'SGD', 'loss_function': 'mse', 'batch_size': 128, 'max_iter': 10000}. Best is trial 1 with value: 0.20302513241767883.


Epoch 1/10000
3953/3953 [==============================] - 10s 2ms/step - loss: 0.6820 - accuracy: 0.5752 - val_loss: 0.6513 - val_accuracy: 0.6501
Epoch 2/10000
3953/3953 [==============================] - 8s 2ms/step - loss: 0.6496 - accuracy: 0.6523 - val_loss: 0.6478 - val_accuracy: 0.6534
Epoch 3/10000
3953/3953 [==============================] - 8s 2ms/step - loss: 0.6470 - accuracy: 0.6537 - val_loss: 0.6456 - val_accuracy: 0.6540
Epoch 4/10000
3953/3953 [==============================] - 8s 2ms/step - loss: 0.6450 - accuracy: 0.6542 - val_loss: 0.6437 - val_accuracy: 0.6543
Epoch 5/10000
3953/3953 [==============================] - 8s 2ms/step - loss: 0.6432 - accuracy: 0.6546 - val_loss: 0.6421 - val_accuracy: 0.6546
Epoch 6/10000
3953/3953 [==============================] - 8s 2ms/step - loss: 0.6416 - accuracy: 0.6547 - val_loss: 0.6406 - val_accuracy: 0.6549
Epoch 7/10000
3953/3953 [==============================] - 9s 2ms/step - loss: 0.6401 - accuracy: 0.6549 - val_loss: 

[I 2023-03-29 22:33:22,221] Trial 9 finished with value: 0.6096231937408447 and parameters: {'hidden_layer_sizes': 32, 'activation': 'tanh', 'hidden_layers': 1, 'output_activation': 'sigmoid', 'alpha': 0.0001, 'optimizer': 'SGD', 'loss_function': 'binary_crossentropy', 'batch_size': 64, 'max_iter': 10000}. Best is trial 1 with value: 0.20302513241767883.


In [7]:
print('Best hyperparameters: ', study.best_params)
print('Best objective value: ', study.best_value)

Best hyperparameters:  {'hidden_layer_sizes': 128, 'activation': 'tanh', 'hidden_layers': 1, 'output_activation': 'sigmoid', 'alpha': 0.0001, 'optimizer': 'Adam', 'loss_function': 'mse', 'batch_size': 128, 'max_iter': 2000}
Best objective value:  0.20302513241767883


#### **Saving study**

In [8]:
import joblib

In [9]:
save_path = './optuna_studies/mlp_study.pkl'

joblib.dump(study, save_path)

['./optuna_studies/mlp_study.pkl']